<a href="https://colab.research.google.com/github/michaelgetachew-abebe/AirBnB_clone/blob/main/ascii_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen-7B",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = False,
    token=userdata.get('HUGGINGFACE_TOKEN')
)

RuntimeError: Unsloth: Failed to load model. Both AutoConfig and PeftConfig loading failed.

AutoConfig error: The repository `Qwen/Qwen-7B` contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Qwen/Qwen-7B.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.

PeftConfig error: Can't find 'adapter_config.json' at 'Qwen/Qwen-7B'

